In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5
# import pytz module for working with time zone
import pytz, os
import numpy as np

# Load env variables
%load_ext dotenv
%dotenv

from stockstats import StockDataFrame

# finRL
from finrl.meta.data_processors.processor_alpaca_2 import AlpacaProcessor2
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame


# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

# Plot with matplotlib
%matplotlib inline

ModuleNotFoundError: No module named 'finrl.meta.data_processors.processor_alpaca_2'

In [2]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# display data on MetaTrader 5 version
print(mt5.version())

(500, 3391, '05 Aug 2022')


In [3]:
# MT5 account infor
MT5_ACCOUNT = os.environ.get('MT5_ACCOUNT')
MT5_PASSWORD = os.environ.get('MT5_PASSWORD')
MT5_SERVER = os.environ.get('MT5_SERVER')

# Alpaca
API_KEY = os.environ.get('API_KEY')
API_SECRET = os.environ.get('API_SECRET')
API_BASE_URL = os.environ.get('API_BASE_URL')
data_url = os.environ.get('data_url')
TIME_INTERVAL_RAW = '1Day'

# Tick info
TICK_START_DATE='2022-07-01'
TICK_END_DATE='2022-07-31'


TIME_INTERVAL = {
    '1Day': mt5.TIMEFRAME_D1,
    '4Hour': mt5.TIMEFRAME_H4,
    '1Hour': mt5.TIMEFRAME_H1,
    '30Min': mt5.TIMEFRAME_M30,
    '15Min': mt5.TIMEFRAME_M15,
    '5Min': mt5.TIMEFRAME_M5,
    '1Min': mt5.TIMEFRAME_M1,
}

tech_indicator_list=[
    "macd",
    "boll_ub",
    "boll_lb",
    "rsi_30",
    "dx_30",
    "close_14_sma", 
    "close_50_sma",
    "close_200_sma",
    "volume_14_sma",
    "volume_xu_volume_14_sma"
]

# TODO: Add XAuUSD' back. It has different length of rows
TIC_LIST = ['EURUSD', 'USDCHF', 'CADCHF']

# MT5 server time: Europe/Moscow  (GMT+3)


## Alpaca

In [4]:
try:
    alpaca_api = tradeapi.REST(API_KEY, API_SECRET, API_BASE_URL, "v2")
except BaseException:
    raise ValueError("Wrong Account Info!")

## MT5

In [5]:
# now connect to another trading account specifying the password
authorized=mt5.login(login=MT5_ACCOUNT, password=MT5_PASSWORD, server=MT5_SERVER)
if authorized:
    # display trading account data 'as is'
    print(mt5.account_info())
    # display trading account data in the form of a list
    print("Show account_info()._asdict():")
    account_info_dict = mt5.account_info()._asdict()
    for prop in account_info_dict:
        print("  {}={}".format(prop, account_info_dict[prop]))
else:
    print("failed to connect at account #{}, error code: {}".format(MT5_ACCOUNT, mt5.last_error()))

failed to connect at account #6106603, error code: (-2, 'Invalid "login" argument')


In [6]:
def fetch_bars(tic_list: [str]) -> pd.DataFrame:
    
    tic_frames = pd.DataFrame()
    
    for tic in tic_list:
        
        # Fetch the bars from MT5
        tic_rates = mt5.copy_rates_range(
                        tic, 
                        TIME_INTERVAL[TIME_INTERVAL_RAW], 
                        datetime.strptime(TICK_START_DATE, '%Y-%m-%d'), 
                        datetime.strptime(TICK_END_DATE, '%Y-%m-%d')
                    )
        bar_frame = pd.DataFrame(tic_rates) 
        
        # Convert timestamp to datetime
        bar_frame['time'] = pd.to_datetime(bar_frame['time'], unit='s')
        
        # Add tick name
        bar_frame['tic'] = tic
        
        # Rename tick_volume to volume
        bar_frame = bar_frame.rename(columns={"time": "date", "tick_volume": "volume"}, errors="raise")
        
        # Drop column "real_volume"
        bar_frame = bar_frame.drop(columns=['real_volume'])
                
        # Concat bars to the main dataframe
        tic_frames = pd.concat([tic_frames, bar_frame])
        
    return tic_frames

In [7]:
tic_frames = fetch_bars(TIC_LIST)
tic_frames 

,date,open,high,low,close,volume,spread,tic
0,2022-07-04,1.04195,1.04622,1.04163,1.04222,93047,10,EURUSD
1,2022-07-05,1.04199,1.04483,1.02346,1.02643,168007,10,EURUSD
2,2022-07-06,1.02641,1.02761,1.01608,1.01842,186102,10,EURUSD
3,2022-07-07,1.01805,1.02206,1.01436,1.01604,146191,10,EURUSD
4,2022-07-08,1.01589,1.01909,1.00711,1.01844,165414,10,EURUSD
5,2022-07-11,1.01733,1.01830,1.00332,1.00387,129851,10,EURUSD
6,2022-07-12,1.00365,1.00734,0.99995,1.00368,168064,10,EURUSD
7,2022-07-13,1.00357,1.01218,0.99974,1.00572,202578,1,EURUSD
8,2022-07-14,1.00545,1.00590,0.99518,1.00201,205682,10,EURUSD
9,2022-07-15,1.00162,1.00976,1.00064,1.00873,154352,10,EURUSD


In [8]:
def add_technical_indicator(
    df: pd.DataFrame,
    tic_list: [str],
    tech_indicator_list=[
        "macd",
        "boll_ub",
        "boll_lb",
        "rsi_30",
        "dx_30",
        "close_14_sma", 
        "close_50_sma",
        "close_200_sma",
        "volume_14_sma",
        "volume_xu_volume_14_sma"
    ],
):
    tic_indicator_frame = pd.DataFrame()

    for tic in tic_list:
        tic_df = df[df['tic'] == tic].copy()
        tic_df = tic_df.sort_values(by=["tic", "date"])

        stock = StockDataFrame.retype(tic_df)
    
        columns = tic_df.columns.values.tolist() + tech_indicator_list
        del columns[0]

        tic_indicator_frame = pd.concat([tic_indicator_frame, stock[columns]])
        tic_indicator_frame.sort_values(by=["date", "tic"])
    
    return tic_indicator_frame.replace(np.nan, 0)

In [9]:
tic_indicator_frame = add_technical_indicator(tic_frames, TIC_LIST)
tic_indicator_frame

,high,low,close,volume,spread,tic,macd,boll_ub,boll_lb,rsi_30,dx_30,close_14_sma,close_50_sma,close_200_sma,volume_14_sma,volume_xu_volume_14_sma
date,,,,,,,,,,,,,,,,
2022-07-04,1.04622,1.04163,1.04222,93047,10,EURUSD,0.000000,0.000000,0.000000,0.000000,0.000000,1.042220,1.042220,1.042220,93047.000000,False
2022-07-05,1.04483,1.02346,1.02643,168007,10,EURUSD,-0.000354,1.056655,1.011995,0.000000,100.000000,1.034325,1.034325,1.034325,130527.000000,True
2022-07-06,1.02761,1.01608,1.01842,186102,10,EURUSD,-0.000700,1.053243,1.004803,0.000000,100.000000,1.029023,1.029023,1.029023,149052.000000,False
2022-07-07,1.02206,1.01436,1.01604,146191,10,EURUSD,-0.000926,1.049434,1.002121,0.000000,100.000000,1.025778,1.025778,1.025778,148336.750000,False
2022-07-08,1.01909,1.00711,1.01844,165414,10,EURUSD,-0.000934,1.045823,1.002797,9.074199,100.000000,1.024310,1.024310,1.024310,151752.200000,True
2022-07-11,1.01830,1.00332,1.00387,129851,10,EURUSD,-0.001544,1.046374,0.995432,5.780204,100.000000,1.020903,1.020903,1.020903,148102.000000,False
2022-07-12,1.00734,0.99995,1.00368,168064,10,EURUSD,-0.001917,1.045092,0.991794,5.752036,100.000000,1.018443,1.018443,1.018443,150953.714286,True
2022-07-13,1.01218,0.99974,1.00572,202578,1,EURUSD,-0.002039,1.043113,0.990592,10.591413,73.131416,1.016852,1.016852,1.016852,157406.750000,False
2022-07-14,1.00590,0.99518,1.00201,205682,10,EURUSD,-0.002277,1.041686,0.988720,9.658396,76.333495,1.015203,1.015203,1.015203,162770.666667,False


In [10]:
tic_indicator_frame[tic_indicator_frame['tic'] == TIC_LIST[0]].index.to_pydatetime().tolist()

[datetime.datetime(2022, 7, 4, 0, 0),
 datetime.datetime(2022, 7, 5, 0, 0),
 datetime.datetime(2022, 7, 6, 0, 0),
 datetime.datetime(2022, 7, 7, 0, 0),
 datetime.datetime(2022, 7, 8, 0, 0),
 datetime.datetime(2022, 7, 11, 0, 0),
 datetime.datetime(2022, 7, 12, 0, 0),
 datetime.datetime(2022, 7, 13, 0, 0),
 datetime.datetime(2022, 7, 14, 0, 0),
 datetime.datetime(2022, 7, 15, 0, 0),
 datetime.datetime(2022, 7, 18, 0, 0),
 datetime.datetime(2022, 7, 19, 0, 0),
 datetime.datetime(2022, 7, 20, 0, 0),
 datetime.datetime(2022, 7, 21, 0, 0),
 datetime.datetime(2022, 7, 22, 0, 0),
 datetime.datetime(2022, 7, 25, 0, 0),
 datetime.datetime(2022, 7, 26, 0, 0),
 datetime.datetime(2022, 7, 27, 0, 0),
 datetime.datetime(2022, 7, 28, 0, 0),
 datetime.datetime(2022, 7, 29, 0, 0)]

In [11]:
def df_to_array(df: pd.DataFrame, tech_indicator_list: [str]) -> pd.DataFrame:
    df = df.copy()
    unique_ticker = df.tic.unique()
    if_first_time = True
    
    for tic in unique_ticker:

        if if_first_time:
            
            price_array = df[df.tic == tic][["close"]].values
            tech_array = df[df.tic == tic][tech_indicator_list].values
#             if if_vix:
#                 turbulence_array = df[df.tic == tic]["VIXY"].values
#             else:
#                 turbulence_array = df[df.tic == tic]["turbulence"].values
            if_first_time = False
    
        else:
    
            price_array = np.hstack(
                [price_array, df[df.tic == tic][["close"]].values]
            )
            tech_array = np.hstack(
                [tech_array, df[df.tic == tic][tech_indicator_list].values]
            )
            
    time_points = df[df['tic'] == TIC_LIST[0]].index.to_pydatetime().tolist()
                        
    print("Successfully transformed into array")
#     return price_array, tech_array, turbulence_array
    return price_array, tech_array, time_points

In [12]:
price_array, tech_array, time_points = df_to_array(tic_indicator_frame,tech_indicator_list)

Successfully transformed into array


In [13]:
time_points

[datetime.datetime(2022, 7, 4, 0, 0),
 datetime.datetime(2022, 7, 5, 0, 0),
 datetime.datetime(2022, 7, 6, 0, 0),
 datetime.datetime(2022, 7, 7, 0, 0),
 datetime.datetime(2022, 7, 8, 0, 0),
 datetime.datetime(2022, 7, 11, 0, 0),
 datetime.datetime(2022, 7, 12, 0, 0),
 datetime.datetime(2022, 7, 13, 0, 0),
 datetime.datetime(2022, 7, 14, 0, 0),
 datetime.datetime(2022, 7, 15, 0, 0),
 datetime.datetime(2022, 7, 18, 0, 0),
 datetime.datetime(2022, 7, 19, 0, 0),
 datetime.datetime(2022, 7, 20, 0, 0),
 datetime.datetime(2022, 7, 21, 0, 0),
 datetime.datetime(2022, 7, 22, 0, 0),
 datetime.datetime(2022, 7, 25, 0, 0),
 datetime.datetime(2022, 7, 26, 0, 0),
 datetime.datetime(2022, 7, 27, 0, 0),
 datetime.datetime(2022, 7, 28, 0, 0),
 datetime.datetime(2022, 7, 29, 0, 0)]

In [14]:
price_array[:, 1]

array([0.96089, 0.96842, 0.97046, 0.97398, 0.97645, 0.98318, 0.98157,
       0.97878, 0.98367, 0.97606, 0.97731, 0.96879, 0.97129, 0.96653,
       0.9608 , 0.96436, 0.96297, 0.95955, 0.95466, 0.95116])

## Train

In [15]:
from finrl.agents.stablebaselines3.models import DRLAgent as DRLAgent_sb3
from finrl.meta.env_stock_trading.env_stocktrading_np_df import StockTradingEnv


ModuleNotFoundError: No module named 'finrl.meta.env_stock_trading.env_stocktrading_np_df'

In [ ]:
TOTAL_TIMESTEPS = 1

PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}

MODEL_NAME = 'ppo'

env_config = {
        "price_array": price_array,
        "tech_array": tech_array,
#         "turbulence_array": turbulence_array,
        "if_train": True,
        "tick_list": TIC_LIST,
        "time_points": time_points
    }
env_instance = StockTradingEnv(config=env_config)

# read parameters
cwd = f'./{MODEL_NAME}'

total_timesteps = TOTAL_TIMESTEPS
agent_params = PPO_PARAMS

In [ ]:
agent = DRLAgent_sb3(env=env_instance)
model = agent.get_model(MODEL_NAME, model_kwargs=agent_params)
trained_model = agent.train_model(
    model=model, tb_log_name=MODEL_NAME, total_timesteps=TOTAL_TIMESTEPS
)
print("Training is finished!")
trained_model.save(cwd)
print("Trained model is saved in " + str(cwd))

In [ ]:
env_instance.render(mode='simple_figure')

## Test

In [ ]:
episode_total_assets = DRLAgent_sb3.DRL_prediction_load_from_file(
            model_name=MODEL_NAME, environment=env_instance, cwd=cwd
        )
episode_total_assets